## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-20-54-10 +0000,bbc,'I'm here to entertain' - Alcaraz books semi-f...,https://www.bbc.com/sport/tennis/articles/c4gz...
1,2025-09-02-20-51-00 +0000,wsj,Opinion | How Tariffs Hurt My American Factory,https://www.wsj.com/opinion/how-tariffs-hurt-m...
2,2025-09-02-20-50-43 +0000,nyt,Trump Signals Imminent Supreme Court Appeal to...,https://www.nytimes.com/2025/09/02/us/politics...
3,2025-09-02-20-49-57 +0000,nyt,Judge Apologizes to Conservative Justices in C...,https://www.nytimes.com/2025/09/02/us/politics...
4,2025-09-02-20-49-00 +0000,wsj,Opinion | Trump’s Disoriented Democrats,https://www.wsj.com/opinion/trumps-disoriented...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,42
139,china,14
847,new,12
223,will,9
7,tariffs,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
142,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,86
53,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...,86
266,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,85
32,2025-09-02-20-04-23 +0000,nypost,Would-be Trump assassin Ryan Routh will be on ...,https://nypost.com/2025/09/02/us-news/would-be...,84
87,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,83


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
142,86,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...
53,44,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...
266,43,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
265,38,2025-09-02-00-13-00 +0000,nypost,"Rep. Jerrold Nadler, 78, will not seek re-elec...",https://nypost.com/2025/09/01/us-news/rep-jerr...
219,37,2025-09-02-07-57-55 +0000,bbc,"Putin, Kim and Xi are meeting in China: BBC co...",https://www.bbc.com/news/videos/cwypyq42e13o?a...
241,34,2025-09-02-03-43-25 +0000,nypost,Chef killed in electric buggy crash after late...,https://nypost.com/2025/09/01/world-news/chef-...
99,34,2025-09-02-17-04-21 +0000,nypost,Ex-NYC mayor endorses Zohran Mamdani for City ...,https://nypost.com/2025/09/02/us-news/ex-nyc-m...
258,33,2025-09-02-01-00-00 +0000,wsj,Disparate factions inside MAHA are sparring ov...,https://www.wsj.com/politics/policy/maha-kenne...
278,32,2025-09-01-22-29-23 +0000,bbc,Watch: Buildings destroyed and rescue efforts ...,https://www.bbc.com/news/videos/cqxgx8d14wwo?a...
214,29,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
